# Loading the ROS node model

First we need to load the ROS node model in Imandra. We do that using `System.mod_use` and load the messaging module and the model module: 

In [1]:
System.mod_use ~quiet:true "src-messages/ros_message.ml";;
System.mod_use ~quiet:true "src-model/ros_model.ml";;

open Ros_model;;
open Ros_message;;

- : unit = ()
- : unit = ()


The model has a `state` datatype and a `one_step : state -> state` transition function.

In [2]:
#show state;;

type nonrec state = {
  mode : Ros_model.mode;
  incoming : Ros_model.incoming option;
  outgoing : Ros_model.outgoing option;
}


The `state` contains two "slots" for incoming and outgoing messages and a `mode` value that describes the current  behaviour mode of our robot:

In [3]:
#show mode;;

type nonrec mode = Stopped | Turning | Driving | Error


Our robot starts in the `Stopped`  mode and then transitions between `Turning` and `Driving` modes, reacting to the sensor inputs. 

In [4]:
one_step;;

- : Ros_model.state -> Ros_model.state = <fun>


# Verifying error mode transition

First thing that we want to verify it that our node will not go into an `Error` mode unless it receives an invalid message.  We can formalize this statement as the following logical expression:

$$ \forall s. ( Mode(s) \ne Error) \,\&\, Valid(IncomingMessage(s)) \,\Rightarrow\, (Mode(one\_step(s)) \ne Error) $$

Meaning that for every state $s$, if the state is not already in the `Error` mode and its incoming message is valid, then the state's mode after we've called `one_step` on it is never the `Error` mode.  

We can almost literally encode this formal excpresson as an Imandra `theorem`:

In [5]:
theorem error_state_invalid_message state =
  match state.incoming with None -> true | Some msg -> 
  (  state.mode <> Error 
  && valid_incoming msg  
  ) ==> (one_step state).mode <> Error


val error_state_invalid_message : Ros_model.state -> bool = <fun>


Proved proof call graph proof summary full graph ground_instances 0 definitions 0 inductions 0 search_time 0.006s details Expand smt_stats (:added-eqs 164
 :conflicts 14
 :datatype-accessor-ax 37
 :datatype-constructor-ax 16
 :datatype-splits 8
 :decisions 21
 :del-clause 19
 :max-memory 16.21
 :memory 10.43
 :minimized-lits 2
 :mk-bool-var 132
 :mk-clause 43
 :num-allocs 527483565
 :num-checks 2
 :propagations 59
 :rlimit-count 67348) Expand start[0.006s]
 if :var_0:.Ros_model.incoming = None then true
 else
 not (:var_0:.Ros_model.mode = Ros_model.Error)
 && (if Is_a(Ros_model.Sensor, Option.get :var_0:.Ros_model.incoming)
 then List.length ….Ros_message.Sensor_msgs.ranges = 5 else true)
 ==> not
 ((if :var_0:.Ros_model.mode = Ros_model.Error
 then
 {Ros_model.mode = :var_0:.Ros_model.mode;
 Ros_model.incoming = :var_0:.Ros_model.incoming;
 Ros_model.outgoing = …}
 else
 if :var_0:.Ros_model.incoming = None
 then
 {Ros_model.mode = :var_0:.Ros_model.mode;
 Ros_model.incoming = :var_0:.Ros_model.incoming;
 Ros_model.outgoing = …}
 else …).Ros_model.mode
 = Ros_model.Error) simplify into (:var_0:.Ros_model.incoming = None
 || not
 (not (:var_0:.Ros_model.mode = Ros_model.Error)
 && (not Is_a(Ros_model.Sensor, Option.get :var_0:.Ros_model.incoming)
 || List.length
 (Destruct(Ros_model.Sensor, 0,
 Option.get :var_0:.Ros_model.incoming)).Ros_message.Sensor_msgs.ranges
 = 5)))
|| not
 ((if :var_0:.Ros_model.mode = Ros_model.Error
 then
 {Ros_model.mode = :var_0:.Ros_model.mode;
 Ros_model.incoming = :var_0:.Ros_model.incoming;
 Ros_model.outgoing = None}
 else
 if :var_0:.Ros_model.incoming = None
 then
 {Ros_model.mode = :var_0:.Ros_model.mode;
 Ros_model.incoming = :var_0:.Ros_model.incoming;
 Ros_model.outgoing = None}
 else
 if not Is_a(Ros_model.Sensor, Option.get :var_0:.Ros_model.incoming)
 || List.length
 (Destruct(Ros_model.Sensor, 0,
 Option.get :var_0:.Ros_model.incoming)).Ros_message.Sensor_msgs.ranges
 = 5
 then … else …).Ros_model.mode
 = Ros_model.Error) expansions [] rewrite_steps forward_chaining unsat (let ((a!1 (asserted (not (= (Ros_model.mode_2128 state_2207)
 Ros_model… digraph "proof" {
p_199 [label="Start (if :var_0:.Ros_model.incoming = None then true\l else\l not (:var_0:.Ros_model.mode = Ros_model.Error)\l && (if Is_a(Ros_model.Sensor, Option.get :var_0:.Ros_model.incoming)\l then List.length ….Ros_message.Sensor_msgs.ranges = 5\l else true)\l ==\> not\l ((if :var_0:.Ros_model.mode = Ros_model.Error\l then\l \{Ros_model.mode = :var_0:.Ros_model.mode;\l Ros_model.incoming = :var_0:.Ros_model.incoming;\l Ros_model.outgoing = …\}\l else\l if :var_0:.Ros_model.incoming = None\l then\l \{Ros_model.mode = :var_0:.Ros_model.mode;\l Ros_model.incoming = :var_0:.Ros_model.incoming;\l Ros_model.outgoing = …\}\l else …).Ros_model.mode\l = Ros_model.Error)\l :time 0.006s)",shape=box,style=filled,fontname="courier",fontsize=14];
p_199 -> p_198 [label=""];
p_198 [label="Simplify ((:var_0:.Ros_model.incoming = None\l \|\| not\l (not (:var_0:.Ros_model.mode = Ros_model.Error)\l && (not\l Is_a(Ros_model.Sensor,\l Option.get :var_0:.Ros_model.incoming)\l \|\| List.length\l (Destruct(Ros_model.Sensor, 0,\l Option.get :var_0:.Ros_model.incoming)).Ros_message.Sensor_msgs.ranges\l = 5)))\l \|\| not\l ((if :var_0:.Ros_model.mode = Ros_model.Error\l then\l \{Ros_model.mode = :var_0:.Ros_model.mode;\l Ros_model.incoming = :var_0:.Ros_model.incoming;\l Ros_model.outgoing = None\}\l else\l if :var_0:.Ros_model.incoming = None\l then\l \{Ros_model.mode = :var_0:.Ros_model.mode;\l Ros_model.incoming = :var_0:.Ros_model.incoming;\l Ros_model.outgoing = None\}\l else\l if not\l Is_a(Ros_model.Sensor,\l Option.get :var_0:.Ros_model.incoming)\l \|\| List.length\l (Destruct(Ros_model.Sensor, 0,\l Option.get :var_0:.Ros_model.incoming)).Ros_message.Sensor_msgs.ranges\l = 5\l then … else …).Ros_model.mode\l = Ros_model.Error)\l :expansions [] :rw [] :fc [])",shape=box,style=filled,fontname="courier",fontsize=14];
p_198 -> p_197 [label=""];

One can see that the theorem is "Proven", meaning that the 

# Verifying stopping if facing a wall

Let us verify some more complex statement about our model -- lets prove that if the robot senses that it is too close to the wall, then it always attemps to either stop or backpedal from it. More precisely, if the front-facing sensor shows a range that is too close (say, less than 0.5 m), then our node always sends out a `Twist` message with `linear.x` velocity that is less or equal to zero.

We create two helper functions that get the range of the front sensor and check that the outgoing message is a `Twist` message and its velocity is not positive:

In [6]:
let get_front_sensor_range state =
  match state.incoming with 
  | Some (Sensor sensor) -> List.nth 2 sensor.Sensor_msgs.ranges 
  | _ -> None 

let no_positive_outgoing_velocity state =
  match state.outgoing with 
  | Some (Twist twist) -> twist.Geometry_msg.linear.x <= 0
  | _ -> false

val get_front_sensor_range : Ros_model.state -> Ros_message.number option =
  <fun>
val no_positive_outgoing_velocity : Ros_model.state -> bool = <fun>


Now we can attempt to prove our goal. We require that the state incoming message is a valid `Sensor` message with the front sensor range < 0.5 m, and we want to prove that after we apply `one_call` to the state -- there is always a `Twist` message with non-positive velocity:

In [7]:
theorem wrong_stop_if_front_sensor_close state =
  match state.incoming with None -> true | Some msg -> 
  match get_front_sensor_range state with None -> true | Some range_front ->
  let state' = one_step state in
  (  range_front < 500
  && valid_incoming msg  
  ) ==> 
  no_positive_outgoing_velocity state' 

val wrong_stop_if_front_sensor_close : Ros_model.state -> bool = <fun>
module CX : sig val state : Ros_model.state end
Error: wrong_stop_if_front_sensor_close is not a theorem.


Refuted call graph proof-attempt digraph "call graph" {
goal [label="not\l(((state.Ros_model.incoming = None\l \|\| (if Is_a(Ros_model.Sensor, Option.get state.Ros_model.incoming)\l && Is_a(Some, state.Ros_model.incoming)\l then\l List.nth 2\l (Destruct(Ros_model.Sensor, 0, Option.get state.Ros_model.incoming)).Ros_message.Sensor_msgs.ranges\l else None)\l = None)\l \|\| not\l (not\l (500 \<=\l Option.get\l (if Is_a(Ros_model.Sensor, Option.get state.Ros_model.incoming)\l && Is_a(Some, state.Ros_model.incoming)\l then\l List.nth 2\l (Destruct(Ros_model.Sensor, 0, Option.get state.Ros_model.incoming)).Ros_message.Sensor_msgs.ranges\l else None))\l && (not Is_a(Ros_model.Sensor, Option.get state.Ros_model.incoming)\l \|\| List.length\l (Destruct(Ros_model.Sensor, 0,\l Option.get state.Ros_model.incoming)).Ros_message.Sensor_msgs.ranges\l = 5)))\l \|\| (Is_a(Ros_model.Twist,\l Option.get\l (if state.Ros_model.mode = Ros_model.Error then … else …).Ros_model.outgoing)\l && Is_a(Some,\l (if state.Ros_model.mode = Ros_model.Error then … else …).Ros_model.outgoing))\l && (Destruct(Ros_model.Twist, 0,\l Option.get\l (if state.Ros_model.mode = Ros_model.Error then … else …).Ros_model.outgoing)).Ros_message.Geometry_msg.linear.Ros_message.x\l \<= 0)",shape=box,style=filled,color="cyan",fontname="courier",fontsize=14];
call_4136 [label="List.length\l(List.tl\l (List.tl\l (List.tl\l (List.tl\l (Destruct(Ros_model.Sensor, 0, Option.get …)).Ros_message.Sensor_msgs.ranges))))",shape=box,style=filled,color="green",fontname="courier",fontsize=14];
call_4136 -> expr_4529 [label="="];
expr_4529 [label="if List.tl\l (List.tl\l (List.tl\l (List.tl\l (Destruct(Ros_model.Sensor, 0, Option.get ….Ros_model.incoming)).Ros_message.Sensor_msgs.ranges)))\l = []\lthen 0\lelse\l 1\l + List.length\l (List.tl\l (List.tl\l (List.tl\l (List.tl\l (List.tl\l (Destruct(Ros_model.Sensor, 0, Option.get ….Ros_model.incoming)).Ros_message.Sensor_msgs.ranges)))))",shape=box,style=filled,color="grey",fontname="courier",fontsize=14];
expr_4138 -> call_4136 [label="calls"];
call_3172 [label="List.nth (-1 + 3)\l(List.tl\l (Destruct(Ros_model.Sensor, 0, Option.get …)).Ros_message.Sensor_msgs.ranges)",shape=box,style=filled,color="green",fontname="courier",fontsize=14];
call_3172 -> expr_3506 [label="="];
expr_3506 [label="if List.tl\l (Destruct(Ros_model.Sensor, 0, Option.get ….Ros_model.incoming)).Ros_message.Sensor_msgs.ranges\l = []\lthen None\lelse\lif -1 + 3 = 0\lthen\l Some\l (List.hd\l (List.tl\l (Destruct(Ros_model.Sensor, 0, Option.get ….Ros_model.incoming)).Ros_message.Sensor_msgs.ranges))\lelse\l List.nth (-1 + -1 + 3)\l (List.tl\l (List.tl\l (Destruct(Ros_model.Sensor, 0, Option.get ….Ros_model.incoming)).Ros_message.Sensor_msgs.ranges))",shape=box,style=filled,color="grey",fontname="courier",fontsize=14];
expr_3175 -> call_3172 [label="calls"];
call_3688 [label="List.length\l(List.tl\l (List.tl\l (Destruct(Ros_model.Sensor, 0, Option.get …)).Ros_message.Sensor_msgs.ranges))",shape=box,style=filled,color="green",fontname="courier",fontsize=14];
call_3688 -> expr_3989 [label="="];
expr_3989 [label="if List.tl\l (List.tl\l (Destruct(Ros_model.Sensor, 0, Option.get ….Ros_model.incoming)).Ros_message.Sensor_msgs.ranges)\l = []\lthen 0\lelse\l 1\l + List.length\l (List.tl\l (List.tl\l (List.tl\l (Destruct(Ros_model.Sensor, 0, Option.get ….Ros_model.incoming)).Ros_message.Sensor_msgs.ranges)))",shape=box,style=filled,color="grey",fontname="courier",fontsize=14];
expr_3690 -> call_3688 [label="calls"];
call_1318 [label="List.nth 1\l(Destruct(Ros_model.Sensor, 0, Option.get …)).Ros_message.Sensor_msgs.ranges",shape=box,style=filled,color="green",fontname="courier",fontsize=14];
call_1318 -> expr_3284 [label="="];
expr_3284 [label="if (Destruct(Ros_model.Sensor, 0, Option.get ….Ros_model.incoming)).Ros_message.Sensor_msgs.ranges\l = []\lthen None\lelse\lif 1 = 0\lthen\l Some\l (List.hd\l (Destruct(Ros_model.Sensor, 0, Option.get ….Ros_model.incoming)).Ros_message.Sensor_m

In [31]:
instance (fun state -> match state.incoming with None -> true | Some msg -> 
  match get_front_sensor_range state with None -> true | Some range_front ->
  let state' = one_step state in
  (  range_front < 500
  && valid_incoming msg  
  ) ==> 
  no_positive_outgoing_velocity state' )

- : Ros_model.state -> bool = <fun>
module CX : sig val state : Ros_model.state end
[✔] Instance found.


Refuted call graph proof-attempt digraph "call graph" {
goal [label="((state.Ros_model.incoming = None\l \|\| (if Is_a(Ros_model.Sensor, Option.get state.Ros_model.incoming)\l && Is_a(Some, state.Ros_model.incoming)\l then\l List.nth 2\l (Destruct(Ros_model.Sensor, 0, Option.get state.Ros_model.incoming)).Ros_message.Sensor_msgs.ranges\l else None)\l = None)\l \|\| not\l (not\l (500 \<=\l Option.get\l (if Is_a(Ros_model.Sensor, Option.get state.Ros_model.incoming)\l && Is_a(Some, state.Ros_model.incoming)\l then\l List.nth 2\l (Destruct(Ros_model.Sensor, 0, Option.get state.Ros_model.incoming)).Ros_message.Sensor_msgs.ranges\l else None))\l && (not Is_a(Ros_model.Sensor, Option.get state.Ros_model.incoming)\l \|\| List.length\l (Destruct(Ros_model.Sensor, 0,\l Option.get state.Ros_model.incoming)).Ros_message.Sensor_msgs.ranges\l = 5)))\l\|\| (Is_a(Ros_model.Twist,\l Option.get\l (if state.Ros_model.mode = Ros_model.Error then … else …).Ros_model.outgoing)\l && Is_a(Some,\l (if state.Ros_model.mode = Ros_model.Error then … else …).Ros_model.outgoing))\l && (Destruct(Ros_model.Twist, 0,\l Option.get\l (if state.Ros_model.mode = Ros_model.Error then … else …).Ros_model.outgoing)).Ros_message.Geometry_msg.linear.Ros_message.x\l \<= 0",shape=box,style=filled,color="cyan",fontname="courier",fontsize=14];
call_2777 [label="List.nth 1\l(Destruct(Ros_model.Sensor, 0, Option.get …)).Ros_message.Sensor_msgs.ranges",shape=box,style=filled,color="green",fontname="courier",fontsize=14];
call_2777 -> expr_11618 [label="="];
expr_11618 [label="if (Destruct(Ros_model.Sensor, 0, Option.get ….Ros_model.incoming)).Ros_message.Sensor_msgs.ranges\l = []\lthen None\lelse\lif 1 = 0\lthen\l Some\l (List.hd\l (Destruct(Ros_model.Sensor, 0, Option.get ….Ros_model.incoming)).Ros_message.Sensor_msgs.ranges)\lelse\l List.nth (-1 + 1)\l (List.tl\l (Destruct(Ros_model.Sensor, 0, Option.get ….Ros_model.incoming)).Ros_message.Sensor_msgs.ranges)",shape=box,style=filled,color="grey",fontname="courier",fontsize=14];
goal -> call_2777 [label="calls"];
call_911 [label="List.nth 3\l(Destruct(Ros_model.Sensor, 0, Option.get …)).Ros_message.Sensor_msgs.ranges",shape=box,style=filled,color="green",fontname="courier",fontsize=14];
call_911 -> expr_11486 [label="="];
expr_11486 [label="if (Destruct(Ros_model.Sensor, 0, Option.get ….Ros_model.incoming)).Ros_message.Sensor_msgs.ranges\l = []\lthen None\lelse\lif 3 = 0\lthen\l Some\l (List.hd\l (Destruct(Ros_model.Sensor, 0, Option.get ….Ros_model.incoming)).Ros_message.Sensor_msgs.ranges)\lelse\l List.nth (-1 + 3)\l (List.tl\l (Destruct(Ros_model.Sensor, 0, Option.get ….Ros_model.incoming)).Ros_message.Sensor_msgs.ranges)",shape=box,style=filled,color="grey",fontname="courier",fontsize=14];
goal -> call_911 [label="calls"];
call_9742 [label="List.nth 2\l(Destruct(Ros_model.Sensor, 0, Option.get …)).Ros_message.Sensor_msgs.ranges",shape=box,style=filled,color="green",fontname="courier",fontsize=14];
call_9742 -> expr_11756 [label="="];
expr_11756 [label="if (Destruct(Ros_model.Sensor, 0, Option.get ….Ros_model.incoming)).Ros_message.Sensor_msgs.ranges\l = []\lthen None\lelse\lif 2 = 0\lthen\l Some\l (List.hd\l (Destruct(Ros_model.Sensor, 0, Option.get ….Ros_model.incoming)).Ros_message.Sensor_msgs.ranges)\lelse\l List.nth (-1 + 2)\l (List.tl\l (Destruct(Ros_model.Sensor, 0, Option.get ….Ros_model.incoming)).Ros_message.Sensor_msgs.ranges)",shape=box,style=filled,color="grey",fontname="courier",fontsize=14];
goal -> call_9742 [label="calls"];
call_11616 [label="List.nth (-1 + 1)\l(List.tl\l (Destruct(Ros_model.Sensor, 0, Option.get …)).Ros_message.Sensor_msgs.ranges)",shape=box,style=filled,color="yellow",fontname="courier",fontsize=14];
expr_11618 -> call_11616 [label="calls"];
call_11668 [label="List.nth (-1 + 2)\l(List.tl\l (Destruct(Ros_model.Sensor, 0, Option.get …)).Ros_message.Sensor_msgs.ranges)",shape=box,style=filled,color="yellow",fontname="courier",fontsize=14];
expr_11756 -> call_11668 [label="calls"];
call_11

In [8]:
CX.state

File "jupyter cell 8", line 1, characters 0-8:
Error: Unbound module CX


We've failed to prove the theorem, and Imandra gave us a conunter example `CX` module. This module contains a state, that refutes the statement of the theorem we've attempted to prove. Lets check the outgoing message if we pass the state into the `one_step`:

In [29]:
CX.state

File "jupyter cell 29", line 1, characters 0-8:
Error: Unbound module CX


In [20]:
theorem stop_if_front_sensor_close state =
  match state.incoming with None -> true | Some msg -> 
  match get_front_sensor_range state with None -> true | Some range_front ->
  let state' = one_step state in
  (  range_front < 500 
  && state.mode <> Error
  && valid_incoming msg  
  ) ==> 
  no_positive_outgoing_velocity state' 

val stop_if_front_sensor_close : Ros_model.state -> bool = <fun>


Proved proof call graph proof summary full graph ground_instances 14 definitions 0 inductions 0 search_time 0.024s details Expand smt_stats (:added-eqs 1541
 :arith-add-rows 17
 :arith-assert-diseq 24
 :arith-assert-lower 72
 :arith-assert-upper 28
 :arith-bound-prop 1
 :arith-eq-adapter 29
 :arith-fixed-eqs 11
 :arith-pivots 9
 :conflicts 60
 :datatype-accessor-ax 98
 :datatype-constructor-ax 149
 :datatype-occurs-check 5329
 :datatype-splits 245
 :decisions 733
 :del-clause 189
 :final-checks 46
 :max-memory 16.80
 :memory 16.80
 :minimized-lits 3
 :mk-bool-var 689
 :mk-clause 207
 :num-allocs 574346420
 :num-checks 30
 :propagations 722
 :rlimit-count 98343) Expand start[0.024s]
 if :var_0:.Ros_model.incoming = None then true
 else
 if (if Is_a(Ros_model.Sensor, Option.get :var_0:.Ros_model.incoming)
 && Is_a(Some, :var_0:.Ros_model.incoming)
 then List.nth 2 ….Ros_message.Sensor_msgs.ranges else None)
 = None
 then true
 else
 Option.get
 (if Is_a(Ros_model.Sensor, Option.get :var_0:.Ros_model.incoming)
 && Is_a(Some, :var_0:.Ros_model.incoming)
 then List.nth 2 ….Ros_message.Sensor_msgs.ranges else None)
 < 500
 && not (:var_0:.Ros_model.mode = Ros_model.Error)
 && (if Is_a(Ros_model.Sensor, Option.get :var_0:.Ros_model.incoming)
 then List.length ….Ros_message.Sensor_msgs.ranges = 5 else true)
 ==> (if Is_a(Ros_model.Twist, Option.get …) && Is_a(Some, …)
 then ….Ros_message.x <= 0 else false) simplify into ((:var_0:.Ros_model.incoming = None
 || (if Is_a(Ros_model.Sensor, Option.get :var_0:.Ros_model.incoming)
 && Is_a(Some, :var_0:.Ros_model.incoming)
 then
 List.nth 2
 (Destruct(Ros_model.Sensor, 0, Option.get :var_0:.Ros_model.incoming)).Ros_message.Sensor_msgs.ranges
 else None)
 = None)
 || not
 ((not
 (500 <=
 Option.get
 (if Is_a(Ros_model.Sensor, Option.get :var_0:.Ros_model.incoming)
 && Is_a(Some, :var_0:.Ros_model.incoming)
 then
 List.nth 2
 (Destruct(Ros_model.Sensor, 0,
 Option.get :var_0:.Ros_model.incoming)).Ros_message.Sensor_msgs.ranges
 else None))
 && not (:var_0:.Ros_model.mode = Ros_model.Error))
 && (not Is_a(Ros_model.Sensor, Option.get :var_0:.Ros_model.incoming)
 || List.length
 (Destruct(Ros_model.Sensor, 0,
 Option.get :var_0:.Ros_model.incoming)).Ros_message.Sensor_msgs.ranges
 = 5)))
|| (Is_a(Ros_model.Twist,
 Option.get
 (if :var_0:.Ros_model.mode = Ros_model.Error then … else …).Ros_model.outgoing)
 && Is_a(Some,
 (if :var_0:.Ros_model.mode = Ros_model.Error then … else …).Ros_model.outgoing))
 && (Destruct(Ros_model.Twist, 0,
 Option.get
 (if :var_0:.Ros_model.mode = Ros_model.Error then …
 else …).Ros_model.outgoing)).Ros_message.Geometry_msg.linear.Ros_message.x
 <= 0 expansions [] rewrite_steps forward_chaining unroll expr (let ((a!1 (Ros_message.Sensor_msgs.ranges_2408
 (get.Ros_model.Sensor.0_2499
 … expansions unroll expr (let ((a!1 (Ros_message.Sensor_msgs.ranges_2408
 (get.Ros_model.Sensor.0_2499
 … expansions unroll expr (let ((a!1 (Ros_message.Sensor_msgs.ranges_2408
 (get.Ros_model.Sensor.0_2499
 … expansions unroll expr (let ((a!1 (Ros_message.Sensor_msgs.ranges_2408
 (get.Ros_model.Sensor.0_2499
 … expansions unroll expr (let ((a!1 (Ros_message.Sensor_msgs.ranges_2408
 (get.Ros_model.Sensor.0_2499
 … expansions unroll expr (let ((a!1 (Ros_message.Sensor_msgs.ranges_2408
 (get.Ros_model.Sensor.0_2499
 … expansions unroll expr (let ((a!1 (Ros_message.Sensor_msgs.ranges_2408
 (get.Ros_model.Sensor.0_2499
 … expansions unroll expr (let ((a!1 (Ros_message.Sensor_msgs.ranges_2408
 (get.Ros_model.Sensor.0_2499
 … expansions unroll expr (let ((a!1 (Ros_message.Sensor_msgs.ranges_2408
 (get.Ros_model.Sensor.0_2499
 … expansions unroll expr (let ((a!1 (Ros_message.Sensor_msgs.ranges_2408
 (get.Ros_model.Sensor.0_2499
 … expansions unroll expr (let ((a!1 (Ros_message.Sensor_msgs.ranges_2408
 (get.Ros_model.Sensor.0_2499
 … expansions unroll expr (let ((a!1 (Ros_message.Sensor_msgs.ranges_2408
 (get.Ros_model.Sensor.0_2499
 … expansions unroll expr (let ((a!1 (Ros_message.Sensor_msgs.ran